# Text Classification

### We will explore Text Classification using nltk, scikit-learn, and gensim

We will use a newsgroups dataset: https://scikit-learn.org/stable/datasets/index.html#the-20-newsgroups-text-dataset (more than 18000 newsgroup posts, across 20 news categories)

#### Goal: Build ML models that predict the category of a newsgroup post, based on the text of the post

Prof Evan Katsamakas,
Gabelli School, 2/2019


In [1]:
import numpy as np
import pandas as pd  
import nltk
import gensim 
from sklearn.datasets import fetch_20newsgroups 
from sklearn.model_selection import train_test_split

# Get the data and take a look

In [2]:
categories = ['talk.politics.guns','rec.sport.baseball'] # We focus on 2 news categories
def get_data():
    data = fetch_20newsgroups(subset='all',
                              shuffle=True,
                              categories=categories,
                              remove=('headers', 'footers', 'quotes'))
    return data

In [3]:
# get text data and their labels
dataset = get_data()
print(dataset.target_names)

corpus, labels = dataset.data, dataset.target

print('Sample document:', corpus[10])
print('Class label:',labels[10])
print('Actual class label:', dataset.target_names[labels[10]])

# split training dataset and testing dataset
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus,labels,test_size=0.3)

['rec.sport.baseball', 'talk.politics.guns']
Sample document: For those who didn't figure it out, the below message was a reply to another
in sci.crypt, for which the poster put t.p.g. in the Followup-To line. I
didn't notice that. Apologies to those who were confused.

The substance makes little sense unless one reads the prior messages.

However, I don't wish to enter into this discussion here, as it will be yet
another rehearsal of a long-tired set of arguments. Suffice it to say that I
disagree both with the interpretation of "well-regulated" in the Second
Amendment offered by gun lovers, and what I think to be their distortion of
the same phrase in the associated Federalist papers. My Webster and my
reading of the language convinces me that the word meant both under control,
and disciplined, and not 'of good marksmanship'. I think the latter a
special interest pleading. No one has yet shown a contemporateous reference
in which "well regulated" unambiguously meant 'of good marksman

# Prepape features for ML 
### {bow, tfidf, word2vec}

In [4]:
#bow features
from sklearn.feature_extraction.text import CountVectorizer #tokenizes and counts words

# build bag of words features' vectorizer and get features
bow_vectorizer=CountVectorizer(min_df=1, ngram_range=(1,1))
bow_train_features = bow_vectorizer.fit_transform(train_corpus)
bow_test_features = bow_vectorizer.transform(test_corpus) 

In [5]:
# tfidf features
from sklearn.feature_extraction.text import TfidfVectorizer #alternatively, use TfidfTransformer()

tfidf_vectorizer=TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
tfidf_train_features = tfidf_vectorizer.fit_transform(train_corpus)  
tfidf_test_features = tfidf_vectorizer.transform(test_corpus)    

In [7]:
# tokenize documents for word2vec
tokenized_train = [nltk.word_tokenize(text)
                   for text in train_corpus]
tokenized_test = [nltk.word_tokenize(text)
                   for text in test_corpus]  

In [8]:
# build word2vec model                   
wv_model = gensim.models.Word2Vec(tokenized_train,
                               size=200,                          #set the size or dimension for the word vectors 
                               window=60,                        #specify the length of the window of words taken as context
                               min_count=10)                   #ignores all words with total frequency lower than                     

In [9]:
def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector 
   

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [10]:
# averaged word vector features from word2vec
avg_wv_train_features = averaged_word_vectorizer(corpus=tokenized_train,
                                                 model=wv_model,
                                                 num_features=200)                   
avg_wv_test_features = averaged_word_vectorizer(corpus=tokenized_test,
                                                model=wv_model,
                                                num_features=200) 

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


# Define metrics for evaluation

In [11]:
from sklearn import metrics

# define a function to evaluate our classification models based on four metrics
def get_metrics(true_labels, predicted_labels):
    
    print ('Accuracy:', np.round(                                                    
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        2))
    print ('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels),
                        2))
    print ('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels),
                        2))
    print ('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels),
                        2))
                        

# Define how to train and evaluate classifier

In [12]:
# define a function that trains the model, performs predictions and evaluates the predictions
def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    # evaluate model prediction performance   
    get_metrics(true_labels=test_labels, 
                predicted_labels=predictions)
    return predictions    

# Train and evaluate {mnb, svm} with bow features

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

mnb = MultinomialNB()
svm = SGDClassifier(loss='hinge', max_iter=100)

# Multinomial Naive Bayes with bag of words features
mnb_bow_predictions = train_predict_evaluate_model(classifier=mnb,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels)

Accuracy: 0.96
Precision: 0.97
Recall: 0.94
F1 Score: 0.96


In [14]:
#Support Vector Machine with bag of words features
svm_bow_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels)

Accuracy: 0.92
Precision: 0.96
Recall: 0.88
F1 Score: 0.92


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


# Train and evaluate {svm} with {tfidf, word2vec} features


In [15]:
# Support Vector Machine with tfidf features
svm_tfidf_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

Accuracy: 0.93
Precision: 0.98
Recall: 0.88
F1 Score: 0.93


In [16]:
# Support Vector Machine with averaged word vector features
svm_avgwv_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=avg_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features,
                                           test_labels=test_labels)

Accuracy: 0.77
Precision: 0.7
Recall: 0.93
F1 Score: 0.8


## CONFUSION MATRIX (for SVM TFIDF) 

In [17]:
# build confusion matrix for SVM TF-IDF-based model
cm = metrics.confusion_matrix(test_labels, svm_avgwv_predictions)
pd.DataFrame(cm, index=range(0,2), columns=range(0,2))  

,0,1
0,167,115
1,19,271


In [19]:
# Observe false positive output
class_names = dataset.target_names
print (class_names[0], '->', class_names[1])

rec.sport.baseball -> talk.politics.guns


In [20]:
# Look at some misclassified documents in detail
import re

num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
    if label == 0 and predicted_label == 1:
        print ('Actual Label:', class_names[label])
        print ('Predicted Label:', class_names[predicted_label])
        print ('Document:-')
        print (re.sub('\n', ' ', document))
        num += 1
        if num == 4:
            break

Actual Label: rec.sport.baseball
Predicted Label: talk.politics.guns
Document:-
   Well Sherri, I'd agree with you except that most 'kiddies' have more sense than to spew their obscenities in front of a group of adults..  I try to edit this newsgroup and feed it to one of the local elementary schools, they have a group of students that just love baseball and are learning to use computers, but I'm telling you, it's gotten to the point that I don't even edit the files anymore, just read them and throw out the trash...  And thanks to all you people that think it's wonderful to include a swear word or two in your signature files, that's really nice...  I have to read the whole article and then toss it out because of the .sig.  Don't get me wrong, I know all the words you do, (and I've even made up some of my own!) or I wouldn't be able to edit them out ;^) but this just doesn't seem to be the place, a public forum, to spew foul language, sorry..  Thanks to all you people that keep in mind,

# Another Model: Desicion Tree

1. Train and evaluate {svm} by using decision tree model
2. CONFUSION MATRIX

In [21]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()
dt_avgwv_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=avg_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features,
                                           test_labels=test_labels)

Accuracy: 0.82
Precision: 0.79
Recall: 0.87
F1 Score: 0.83


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [22]:
cm_dt = metrics.confusion_matrix(test_labels, dt_avgwv_predictions)
pd.DataFrame(cm, index=range(0,2), columns=range(0,2))  

,0,1
0,278,4
1,34,256
